In [1]:
import argparse
from dataclasses import dataclass
import json
import logging
import os
import random
import sys
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import transformers
import wandb
from tqdm.auto import tqdm, trange


logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s: %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [2]:
@dataclass
class CustomArguments(transformers.TrainingArguments):
    sample_train: int = 0
    sample_eval: int = 0
    num_choices: int = 0
    model_name_or_path: str = "asdf"  # this is no longer a TrainingArgument attribute
        
    # python dataclasses cannot have positional attributes in subclass,
    # so give all attributes defaults and then make sure they are changed
    def __post_init__(self):
        if not (self.sample_train * self.sample_eval * self.num_choices) or \
               self.model_name_or_path == "asdf":  # make sure none are still default value
            raise TypeError("__init__ missing required argument(s)")

def get_args():
    """ Set hyperparameters """
    args = CustomArguments(
        output_dir="checkpoint",
        model_name_or_path="roberta-base",
        overwrite_output_dir=True,
        do_train=False,  # Zero shot
        do_eval=True,
        per_device_eval_batch_size=8,
        learning_rate=1e-5,  # Should not matter because not training
        weight_decay=0.1,
        save_total_limit=2,
        seed=123,
        sample_train=200,
        sample_eval=-1,
        num_choices=2,
    )
    
    return args

In [3]:
def get_data(file_path, sample, num_choices):
    data_file = open(file_path, "r")
    logger.info("Reading QA instances from jsonl dataset at: %s", file_path)
    item_jsons = []
    item_ids = []
    questions = []
    choice_lists = []
    answer_ids = []
    for line in data_file:
        item_jsons.append(json.loads(line.strip()))

    if sample != -1:
        item_jsons = random.sample(item_jsons, sample)
        logger.info("Sampling %d examples", sample)

    for item_json in tqdm(item_jsons,total=len(item_jsons)):
        item_id = item_json["id"]

        question_text = item_json["question"]["stem"]

        choice_label_to_id = {}
        choice_text_list = []
        choice_context_list = []
        choice_label_list = []
        choice_annotations_list = []

        any_correct = False
        choice_id_correction = 0

        for choice_id, choice_item in enumerate(item_json["question"]["choices"]):
            choice_label = choice_item["label"]
            choice_label_to_id[choice_label] = choice_id - choice_id_correction
            choice_text = choice_item["text"]

            choice_text_list.append(choice_text)
            choice_label_list.append(choice_label)

            if item_json.get('answerKey') == choice_label:
                if any_correct:
                    raise ValueError("More than one correct answer found for {item_json}!")
                any_correct = True


        if not any_correct and 'answerKey' in item_json:
            raise ValueError("No correct answer found for {item_json}!")


        answer_id = choice_label_to_id.get(item_json.get("answerKey"))
        # Pad choices with empty strings if not right number
        if len(choice_text_list) != num_choices:
            choice_text_list = (choice_text_list + num_choices * [''])[:num_choices]
            choice_context_list = (choice_context_list + num_choices * [None])[:num_choices]
            if answer_id is not None and answer_id >= num_choices:
                logging.warning(f"Skipping question with more than {num_choices} answers: {item_json}")
                continue

        item_ids.append(item_id)
        questions.append(question_text)
        choice_lists.append(choice_text_list)
        answer_ids.append(answer_id)

    data_file.close()
    return questions, choice_lists, answer_ids

In [4]:
class BERTDataset(Dataset):
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
        out = tokenizer(questions, max_length=25, padding="max_length")
        self.input_ids = out["input_ids"]
        self.token_type_ids = out["token_type_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "token_type_ids": self.token_type_ids[i],
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }
    

class RoBERTaDataset(Dataset):
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
#         if "t5" in tokenizer.name_or_path.lower():
#             questions = [question.replace('[MASK]', '') for question in questions]
#         else:
        questions = [question.replace('[MASK]', tokenizer.mask_token) for question in questions]
        out = tokenizer(questions, max_length=25, padding="max_length")
        self.input_ids = out["input_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }

In [38]:
def evaluate(args, model, tokenizer, eval_dataset):
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.per_device_eval_batch_size)

    logger.info(f"***** Running evaluation  *****")
    logger.info(f"  Num examples = {len(eval_dataset)}")
    logger.info(f"  Batch size = {args.eval_batch_size}")
    eval_dataloader = tqdm(eval_dataloader, desc="Evaluating")
    
    print(tokenizer.mask_token)
    MASK_ID = tokenizer.encode(tokenizer.mask_token, add_special_tokens=False)
    assert len(MASK_ID) == 1
    MASK_ID = MASK_ID[0]
    if "t5" in args.model_name_or_path.lower():
        LABELS = tokenizer("<extra_id_0> blank <extra_id_1> </s>", add_special_tokens=False, return_tensors="pt")
        LABELS = LABELS.input_ids.cuda()  # from testing, "blank" can be any word and results are the same 
    
    all_answers = []
    all_preds = []
    for batch in eval_dataloader:
        model.eval()
        
        # batch["choice_list"] is [num_choices, batch_size]
        for i in range(len(batch["choice_list"][0])):
            all_answers.append(batch["choice_list"][batch["answer_id"][i]][i])
        
        choice_lists = batch.pop("choice_list")
        del batch["answer_id"] 
        for key in batch:
            batch[key] = torch.stack(batch[key], dim=-1).cuda()
        
        
        with torch.no_grad():
            if "gpt" not in args.model_name_or_path.lower():
                if "t5" not in args.model_name_or_path.lower():
                    outputs = model(**batch)
                else:
                    outputs = model(input_ids=batch["input_ids"], labels=LABELS)

                logits = outputs.logits
                choice_ids = []

                for i, logit in enumerate(logits):  # Assuming all are single tokens
                    if "t5" not in args.model_name_or_path.lower():
                        MASK_INDEX = batch["input_ids"][i].tolist().index(MASK_ID)
                    else:
                        MASK_INDEX = 1  # TODO: not hardcode

                    choice_ids = torch.tensor([tokenizer.encode(" " + choice_lists[j][i], add_special_tokens=False)[0] for j in range(len(choice_lists))])
                    probs = logit[MASK_INDEX].index_select(0, choice_ids.cuda())
                    max_ind = torch.argmax(probs)
                    all_preds.append(choice_lists[max_ind][i])
            else:
                
#         else:
#             with torch.no_grad():
#                 outputs = model.generate(input_ids=batch["input_ids"], 
#                           num_beams=200, num_return_sequences=20,
#                           max_length=5)
            
#                 match = False
#                 for seq in outputs:
#                     if match:
#                         break
#                     for i, choice in enumerate(choice_lists):
#                         choice_id = tokenizer.encode(choice[0], add_special_tokens=False)[0]
#                         if choice_id in seq:
#                             all_preds.append(choice[0])
#                             match = True
#                             break

#                 if not match:
#                     all_preds.append(-1)

    return all_answers, all_preds

In [56]:
args = get_args()
'''
"bert-base-uncased"
"distilbert-base-uncased"
"bert-large-uncased"
"bert-large-uncased-whole-word-masking" 
"roberta-large"
"facebook/bart-large"
"t5-large"
"albert-large-v1"
'''

args.model_name_or_path = "facebook/bart-large"
args.num_choices = 2
transformers.set_seed(args.seed)
if "t5" in args.model_name_or_path.lower():
    model = transformers.T5ForConditionalGeneration.from_pretrained(args.model_name_or_path).cuda()
    args.per_device_eval_batch_size = 1
    tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name_or_path)
    tokenizer.mask_token = "<extra_id_0>"
elif "gpt" in args.model_name_or_path.lower():
    model = transformers.AutoModelForMaskedLM.from_pretrained(args.model_name_or_path).cuda()
    args.per_device_eval_batch_size = 1
    tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name_or_path)
    tokenizer.mask_token = "[MASK]"
else:
    model = transformers.AutoModelForMaskedLM.from_pretrained(args.model_name_or_path).cuda()
    tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name_or_path)

In [66]:
'''
"data/number_comparison_age_compare_masked_dev.jsonl"
"data/coffee_cats_quantifiers_dev.jsonl"
"data/size_comparison_dev.jsonl"
"data/antonym_synonym_negation_dev.jsonl"
"data/hypernym_conjunction_dev.jsonl"
'''
train_path = "data/antonym_synonym_negation_dev.jsonl"
args.num_choices = 2
eval_path = train_path #"data/coffee_cats_quantifiers_dev.jsonl"
train_questions, train_choices, train_answer_ids = get_data(train_path, args.sample_train, args.num_choices)
eval_questions, eval_choices, eval_answer_ids = get_data(eval_path, args.sample_eval, args.num_choices)
AgeDataset = RoBERTaDataset if any(prefix in args.model_name_or_path.lower() for prefix in ("roberta", "bart", "distil", "electra", "t5")) else BERTDataset
train_dataset = AgeDataset(train_questions, train_choices, train_answer_ids, tokenizer)
eval_dataset = AgeDataset(eval_questions, eval_choices, eval_answer_ids, tokenizer)

05/31/2021 15:07:20: Reading QA instances from jsonl dataset at: data/antonym_synonym_negation_dev.jsonl
05/31/2021 15:07:20: Sampling 200 examples


  0%|          | 0/200 [00:00<?, ?it/s]

05/31/2021 15:07:20: Reading QA instances from jsonl dataset at: data/antonym_synonym_negation_dev.jsonl


  0%|          | 0/500 [00:00<?, ?it/s]

In [67]:
all_answers, all_preds = evaluate(args, model, tokenizer, eval_dataset)
# correct = [1 if all_answers[i] == all_preds[i] else 0 for i in range(len(all_answers))]
# print(np.array(correct))

05/31/2021 15:07:21: ***** Running evaluation  *****
05/31/2021 15:07:21:   Num examples = 500
05/31/2021 15:07:21:   Batch size = 8


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

<mask>


In [68]:
(np.array(all_answers) == np.array(all_preds)).mean()

0.538

In [35]:
tokenizer.encode("smaller", add_special_tokens=False)

[23115, 254]

In [37]:
tokenizer.decode(254)

'er'

In [9]:
a = 0
b = 0
for i in range(len(all_answers)):
    if all_preds[i] != -1:
        b += 1
        if all_preds[i] == all_answers[i]:
            a += 1

In [10]:
a/b

0.47345132743362833

In [11]:
b/len(all_answers)

0.7545909849749582

In [35]:
plt.scatter(first_age, second_age, c=all_preds, marker="s")
plt.title(f"Age Comparison {args.model_name_or_path}")
plt.axis("square")
plt.xlabel("Age 1")
plt.ylabel("Age 2")
#plt.savefig(f"imgs/{args.model_name_or_path.rsplit('/', 1)[-1]}-ages-double.jpg", bbox_inches="tight")
plt.show()

NameError: name 'first_age' is not defined

In [13]:
tokenizer.decode(eval_dataset[2]["input_ids"])

'<s>The size of a nail is usually much<mask> than the size of a pen.</s><pad><pad><pad><pad><pad><pad><pad>'

In [15]:
all_preds[2]

'smaller'